In [1]:
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
import time
from IPython.display import clear_output
import talib
# from joblib import Parallel, delayed
# from tqdm import tqdm
# import xlwings
# import datetime
# import mibian
# import swifter
# from joblib import Parallel, delayed

In [3]:
inst = pd.DataFrame(kite.instruments("NFO"))
inst_nse = pd.DataFrame(kite.instruments("NSE"))

In [4]:
nifty_token = 256265 # NIFTY 50	
expiry_day = 'NIFTY24215'

expiry_march = (
    inst[(inst.name == "NIFTY") & (inst.tradingsymbol.str.contains(expiry_day))]
    # .query("(strike >= @lower_price )&( strike <= @upper_price)")
    # .assign(strike_type=lambda x: x["strike"].astype(int).astype(str) + x["instrument_type"])
)

In [ ]:
import time 
start_dt = '2024-02-07'
end_dt = '2024-02-16'
time_frame = 'minute'

while True:
    df = pd.DataFrame(kite.historical_data(nifty_token, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))

    # Stochastic Oscillator with specified settings
    df['slowk'], df['slowd'] = talib.STOCH(df['high'], df['low'], df['close'],
                                        fastk_period=7, slowk_period=3, slowk_matype=0,
                                        slowd_period=3, slowd_matype=0)

    # First MACD Indicator with specified settings
    df['macd_13_21'], df['macdsignal_13_21'], df['macdhist_13_21'] = talib.MACD(df['close'],
                                                                                fastperiod=13, slowperiod=21,
                                                                                signalperiod=9)

    # Second MACD Indicator with specified settings
    df['macd_34_150'], df['macdsignal_34_150'], df['macdhist_34_150'] = talib.MACD(df['close'],
                                                                                fastperiod=34, slowperiod=150,
                                                                                signalperiod=9)
    df.dropna(inplace=True)
    # Calculate the crossover of slowk and slowd for Stochastic Oscillator
    df['stoch_cross_up'] = ((df['slowk'] > df['slowd']) & (df['slowk'].shift(1) < df['slowd'].shift(1)))
    df['stoch_cross_down'] = ((df['slowk'] < df['slowd']) & (df['slowk'].shift(1) > df['slowd'].shift(1)))

    # Define the buy and sell conditions
    buy_condition = ((df['macdhist_13_21'] < 0) & (df['macdhist_34_150'] > 0) &
                    (df['stoch_cross_up']) & (df['slowk'] < 20) & (df['slowd'] < 20))

    sell_condition = ((df['macdhist_13_21'] > 0) & (df['macdhist_34_150'] < 0) &
                    (df['stoch_cross_down']) & (df['slowk'] > 80) & (df['slowd'] > 80))

    # Add buy and sell signals to the dataframe
    df['buy_signal'] = 0  # Initialize the column with zeros
    df['sell_signal'] = 0  # Initialize the column with zeros

    # If conditions are met, mark the signal
    df.loc[buy_condition, 'buy_signal'] = 1
    df.loc[sell_condition, 'sell_signal'] = 1

    # Print the buy and sell signals on date 14th February 2024
    df["date_only"] = pd.to_datetime(df.date.dt.date)
    df = df.query("date_only == '2024-02-15'")
    print("buy: ",df.query("buy_signal == 1 "))
    print("\n\n")
    print("sell: ",df.query("sell_signal == 1 "))

    time.sleep(60)
    clear_output(wait=True)



In [10]:
import json
import time

while True:
    # Load the configuration file
    with open('config.json', 'r') as file:
        config = json.load(file)

    # Access the variables from the config
    strike_price_ce = config["strike_price"]["ce"]
    strike_price_pe = config["strike_price"]["pe"]
    symbol_ce = f"NFO:NIFTY24FEB{strike_price_ce}CE"
    symbol_pe = f"NFO:NIFTY24FEB{strike_price_pe}PE"

    support_ce = config["support"]["ce"]
    support_pe = config["support"]["pe"]
    resistance_ce = config["resistance"]["ce"]
    resistance_pe = config["resistance"]["pe"]

    position_call = config["position"]["call"]
    position_put = config["position"]["put"]
    call_side = config["side"]["call"]
    put_side = config["side"]["put"]
    breakout = config["breakout"]
    position_put_breakout = config["position"]["put_breakout"]
    position_call_breakout = config["position"]["call_breakout"]

    range_ce = config["range"]["ce"]
    range_pe = config["range"]["pe"]
    stop_loss_ce = config["stop_loss"]["ce"]
    stop_loss_pe = config["stop_loss"]["pe"]
    breakout_target = config["breakout_target"]
    breakout_stop_loss = config["stop_loss"]["breakout"]
    sl_ce_hit = config["sl_hit"]["ce"]
    sl_pe_hit = config["sl_hit"]["pe"]
    


    print("CE buy and target",support_ce*(1+range_ce), resistance_ce*(1-range_ce),)
    print("PE  buy and target",support_pe*(1+range_pe), resistance_pe*(1-range_pe))
    print("ce stop loss",support_ce*(1-stop_loss_ce))
    print("pe stop loss",support_pe*(1-stop_loss_pe))
    print(call_side, put_side)
    time.sleep(5)
    break

CE buy and target 141.78 153.85999999999999
PE  buy and target 151.98 182.28
ce stop loss 132.04999999999998
pe stop loss 141.54999999999998
True True
